# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,margaret river,-33.9500,115.0667,20.07,78,45,3.32,AU,1732088697
1,1,tiksi,71.6872,128.8694,-29.48,98,51,2.23,RU,1732088697
2,2,port-aux-francais,-49.3500,70.2167,4.34,78,64,14.51,TF,1732088697
3,3,waitangi,-43.9535,-176.5597,14.34,91,73,4.02,NZ,1732088697
4,4,grytviken,-54.2811,-36.5092,3.57,79,24,3.91,GS,1732088697


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map
humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.7,
    color = "City"
)

# Save the map plot
hvplot.save(humidity_map_plot, 'output_data/humidity_map.png')
hvplot.save(humidity_map_plot, 'output_data/humidity_map.html')

# Display the map plot
humidity_map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
46,46,tura,25.5198,90.2201,26.11,50,0,1.62,IN,1732088703
67,67,belmonte,-15.8631,-38.8828,23.36,88,0,2.72,BR,1732088705
135,135,kalaswala,32.2009,74.6491,22.80,16,0,2.46,PK,1732088713
204,204,ujhani,28.0167,79.0167,25.50,28,0,2.29,IN,1732088721
281,281,duba,27.3513,35.6901,24.55,31,0,2.09,SA,1732088729


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
46,tura,IN,25.5198,90.2201,50,
67,belmonte,BR,-15.8631,-38.8828,88,
135,kalaswala,PK,32.2009,74.6491,16,
204,ujhani,IN,28.0167,79.0167,28,
281,duba,SA,27.3513,35.6901,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
       "categories":"accommodation.hotel",
        "apiKey": geoapify_key,
        "limit":1

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
                                
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: Hotel Polo Orchid
belmonte - nearest hotel: No hotel found
kalaswala - nearest hotel: No hotel found
ujhani - nearest hotel: No hotel found
duba - nearest hotel: No hotel found
tinajo - nearest hotel: No hotel found
uttar char fasson - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
46,tura,IN,25.5198,90.2201,50,Hotel Polo Orchid
67,belmonte,BR,-15.8631,-38.8828,88,No hotel found
135,kalaswala,PK,32.2009,74.6491,16,No hotel found
204,ujhani,IN,28.0167,79.0167,28,No hotel found
281,duba,SA,27.3513,35.6901,31,No hotel found
286,tinajo,ES,29.0788,-13.7081,68,No hotel found
294,uttar char fasson,BD,22.2332,90.7132,46,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plotay

hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1.0,
    color = "City",
    hover_cols = ["Country","Hotel Name"]
)

# Save the map plot
hvplot.save(hotel_map_plot, 'output_data/hotel_map.png')
hvplot.save(hotel_map_plot, 'output_data/hotel_map.html')

# Display the map plot
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)